In [0]:
catalog = dbutils.widgets.get("catalog")
schema = dbutils.widgets.get("schema")

In [0]:
%sql
MERGE INTO ${catalog}.${schema}.property_silver AS target
USING (
  SELECT *
  FROM (
    SELECT *, ROW_NUMBER() OVER (PARTITION BY zpid ORDER BY file_name DESC) AS rn
    FROM ${catalog}.${schema}.property_bronze
  ) AS deduped
  WHERE rn = 1
) AS source
ON target.zpid = source.zpid

WHEN MATCHED THEN
  UPDATE SET
    target.address = source.address,
    target.city = source.city,
    target.state = source.state,
    target.zipcode = source.zipcode,
    target.latitude = source.latitude,
    target.longitude = source.longitude,
    target.price = source.price,
    target.zestimate = source.zestimate,
    target.bedrooms = source.bedrooms,
    target.bathrooms = source.bathrooms,
    target.livingAreaValue = source.livingAreaValue,
    target.yearBuilt = source.yearBuilt,
    target.homeType = source.homeType,
    target.description = source.description,
    target.ImgSrc = source.ImgSrc,
    target.agent_name = source.agent_name,
    target.agent_phone = source.agent_phone,
    target.datePosted = to_timestamp(source.datePosted),
    target.dateSold = to_timestamp(source.dateSold),
    target.file_name = source.file_name,
    target.Md5_hash = md5(concat_ws('',source.zpid,source.address,
      source.city, source.state, source.zipcode, source.latitude, source.longitude,
      source.price, source.zestimate, source.bedrooms, source.bathrooms,
      source.livingAreaValue, source.yearBuilt, source.homeType, source.description,
      source.ImgSrc, source.agent_name, source.agent_phone,
      to_timestamp(source.datePosted), to_timestamp(source.dateSold), source.file_name))

WHEN NOT MATCHED THEN 
  INSERT (zpid, address, city, state, zipcode, latitude, longitude, price, zestimate,
    bedrooms, bathrooms, livingAreaValue, yearBuilt, homeType, description,
    ImgSrc, agent_name, agent_phone, datePosted, dateSold, file_name, Md5_hash)
  VALUES (
    source.zpid, source.address, source.city, source.state, source.zipcode,
    source.latitude, source.longitude, source.price, source.zestimate,
    source.bedrooms, source.bathrooms, source.livingAreaValue, source.yearBuilt,
    source.homeType, source.description, source.ImgSrc, source.agent_name,
    source.agent_phone, to_timestamp(source.datePosted), to_timestamp(source.dateSold),
    source.file_name,
    md5(concat_ws('',source.zpid,source.address,source.city,source.state,source.zipcode,
      source.latitude,source.longitude,source.price,source.zestimate,source.bedrooms,
      source.bathrooms,source.livingAreaValue,source.yearBuilt,source.homeType,
      source.description,source.ImgSrc,source.agent_name,source.agent_phone,
      to_timestamp(source.datePosted),to_timestamp(source.dateSold),source.file_name))
  );


In [0]:
%sql
select * from ${catalog}.${schema}.property_silver

In [0]:
dbutils.jobs.taskValues.set(key="error", value=str(e))  # 'e' is the exception